In [1]:
import pandas as pd
df = pd.read_csv("/home/aayush/accelerometer/trainClassification_input.csv")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [15]:
df['label'].unique()

KeyError: 'label'

In [16]:
df.head()

,Unnamed: 0,enmoTrunc,enmoAbs,xMean,yMean,zMean,xRange,yRange,zRange,xStd,...,totalPower,temp,samples,dataErrors,clipsBeforeCalibr,clipsAfterCalibr,rawSamples,participant,MET,annotation
0,0,0.001423,0.003816,-0.166901,-0.484580,0.857529,0.030880,0.031312,0.031563,0.002064,...,-13.849006,0.0,3000,0,0,0,3001,15.0,0.95,sleep
1,1,0.001348,0.003484,-0.178192,-0.483312,0.856078,0.108079,0.062623,0.047344,0.012771,...,-13.512001,0.0,3000,0,0,0,3002,15.0,0.95,sleep
2,2,0.003601,0.007880,-0.154394,-0.423471,0.863931,1.173425,0.438355,0.599680,0.174307,...,-12.608712,0.0,3000,0,0,0,3002,15.0,0.95,sleep
3,3,0.004124,0.007625,0.122261,-0.335273,0.924189,0.926388,0.313111,0.426089,0.134946,...,-13.642994,0.0,3000,0,0,0,3002,15.0,0.95,sleep
4,4,0.001921,0.002880,0.158215,-0.332628,0.930696,0.030880,0.031312,0.031563,0.003275,...,-14.460284,0.0,3000,0,0,0,3002,15.0,0.95,sleep


In [13]:
df.to_csv("/home/aayush/accelerometer/accProcess_original_with_annotation.csv")

In [14]:
df = pd.read_csv("/home/aayush/accelerometer/accProcess_original_with_annotation.csv")

In [55]:
test_features_df = df[df["participant"].isin([15.0, 3.0])].iloc[:, :-4]
test_labels_df = df[df["participant"].isin([15.0, 3.0])]["annotation"]

print(test_features_df)
print(test_labels_df.unique())

       enmoTrunc   enmoAbs     xMean     yMean     zMean    xRange    yRange  \
0       0.001423  0.003816 -0.166901 -0.484580  0.857529  0.030880  0.031312   
1       0.001348  0.003484 -0.178192 -0.483312  0.856078  0.108079  0.062623   
2       0.003601  0.007880 -0.154394 -0.423471  0.863931  1.173425  0.438355   
3       0.004124  0.007625  0.122261 -0.335273  0.924189  0.926388  0.313111   
4       0.001921  0.002880  0.158215 -0.332628  0.930696  0.030880  0.031312   
...          ...       ...       ...       ...       ...       ...       ...   
22044   0.001538  0.003173  0.381244 -0.286702  0.878750  0.031672  0.015713   
22045   0.001451  0.003161  0.381158 -0.287136  0.878465  0.031672  0.015713   
22046   0.001329  0.002995  0.381127 -0.287960  0.878126  0.031672  0.015713   
22047   0.001350  0.002940  0.381185 -0.288220  0.878127  0.031672  0.015713   
22048   0.001283  0.002939  0.381045 -0.288785  0.877854  0.031672  0.015713   

         zRange      xStd      yStd  ..

In [56]:
train_features_df = df[~df["participant"].isin([15.0, 3.0])].iloc[:, :-4]
train_labels_predict = df[~df["participant"].isin([15.0, 3.0])]["annotation"]

print(train_features_df)
print(train_labels_predict)

        enmoTrunc   enmoAbs     xMean     yMean     zMean    xRange    yRange  \
3120     0.000468  0.003644 -0.554207  0.217694 -0.800016  0.030874  0.031292   
3121     0.003088  0.003645 -0.566165  0.217381 -0.798266  0.046311  0.031292   
3122     0.003211  0.003708 -0.567642  0.217162 -0.797516  0.046311  0.031292   
3123     0.003188  0.003617 -0.568156  0.216828 -0.797302  0.030874  0.031292   
3124     0.003038  0.003544 -0.568095  0.216594 -0.797123  0.030874  0.031292   
...           ...       ...       ...       ...       ...       ...       ...   
462472   0.005676  0.005677 -0.673620 -0.325168 -0.672207  0.016061  0.031951   
462473   0.006095  0.006095 -0.673528 -0.325300 -0.672869  0.031749  0.031951   
462474   0.006277  0.006277 -0.673516 -0.325243 -0.673177  0.031377  0.031862   
462475   0.006433  0.006433 -0.673527 -0.325302 -0.673372  0.015689  0.031862   
462476   0.006875  0.006905 -0.673563 -0.325355 -0.673915  0.062754  0.031862   

          zRange      xStd 

In [57]:
def _Model(**kwargs):
    return BalancedRandomForestClassifier(
        n_estimators=1000,
        max_depth=10,
        min_samples_leaf=1,
        replacement=True,
        sampling_strategy='not minority',
        random_state=42,
        **kwargs
    )

In [58]:
from imblearn.ensemble import BalancedRandomForestClassifier

print('Training...')
model = _Model(n_jobs=10, verbose=1)
model = model.fit(train_features_df, train_labels_predict)
model.verbose = 0  # silence future calls to .predict()
labels = model.classes_

Training...


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    2.7s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   14.0s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:   32.3s
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:   57.8s
[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:  1.2min finished


In [59]:
print(labels)
pid = df[~df["participant"].isin([15.0, 3.0])]["participant"].to_numpy()
print(len(pid))
print(len(train_labels_predict))

['bicycling' 'mixed' 'sit-stand' 'sleep' 'vehicle' 'walking']
456117
456117


In [60]:
def saveToTar(tarOut, **kwargs):
    """Save objects to tar file. Objects must be passed as keyworded arguments,
    then the key is used for the object name in the tar file.

    :param **kwargs: Objects to be saved passed as keyworded arguments.

    :return: tar file written to <tarOut>
    :rtype: void
    """

    try:

        tmpdir = tempfile.mkdtemp()

        with tarfile.open(tarOut, mode='w') as tf:

            for key, val in kwargs.items():
                pth = os.path.join(tmpdir, key)
                joblib.dump(val, pth, compress=True)
                tf.add(pth, arcname=key)

        print('Models saved to', tarOut)

    finally:

        try:
            shutil.rmtree(tmpdir)
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))


def getFileFromTar(tarArchive, targetFile):
    """Read file from tar

    This is currently more tricky than it should be see
    https://github.com/numpy/numpy/issues/7989

    :param str tarArchive: Input tarfile object
    :param str targetFile: Target individual file within .tar

    :return: file object byte stream
    :rtype: object
    """

    with tarfile.open(tarArchive, 'r') as t:
        b = BytesIO()
        try:
            b.write(t.extractfile(targetFile).read())
        except KeyError:
            return None
        b.seek(0)

    return b


def trainHMM(Y_prob, Y_true, labels=None, uniform_prior=True):
    """ https://en.wikipedia.org/wiki/Hidden_Markov_model

    :return: Dictionary containing prior, emission and transition
        matrices, and corresponding labels.
    :rtype: dict

    """

    if labels is None:
        labels = np.unique(Y_true)

    if uniform_prior:
        # All labels with equal probability
        prior = np.ones(len(labels)) / len(labels)
    else:
        # Label probability equals empirical rate
        prior = np.mean(Y_true.reshape(-1, 1) == labels, axis=0)

    emission = np.vstack(
        [np.mean(Y_prob[Y_true == label], axis=0) for label in labels]
    )
    transition = np.vstack(
        [np.mean(Y_true[1:][(Y_true == label)[:-1]].reshape(-1, 1) == labels, axis=0)
            for label in labels]
    )

    params = {'prior': prior, 'emission': emission, 'transition': transition, 'labels': labels}

    return params

In [61]:
from sklearn.model_selection import cross_val_predict, cross_val_score
import numpy as np

print('Cross-predicting to derive the observations for HMM...')

NJOBS_PER_CV_MODEL = min(2, 10)
cvp = cross_val_predict(
    _Model(n_jobs=NJOBS_PER_CV_MODEL), train_features_df, train_labels_predict, groups=pid,
    cv=10,
    n_jobs=10 // NJOBS_PER_CV_MODEL,
    method="predict_proba",
    verbose=3,
)

Cross-predicting to derive the observations for HMM...


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   5 out of  10 | elapsed:  2.2min remaining:  2.2min
[Parallel(n_jobs=5)]: Done  10 out of  10 | elapsed:  4.5min finished


In [62]:
labels

array(['bicycling', 'mixed', 'sit-stand', 'sleep', 'vehicle', 'walking'],
      dtype=object)

In [63]:
print('Training HMM...')
train_labels_predict_array = np.array(train_labels_predict)
hmmParams = trainHMM(cvp,  train_labels_predict_array)

Training HMM...


In [64]:
hmmParams

{'prior': array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667]),
 'emission': array([[0.15521078, 0.16979104, 0.17052532, 0.1732728 , 0.16172553,
         0.16947453],
        [0.15623009, 0.16607333, 0.17150606, 0.17277507, 0.16422867,
         0.16918678],
        [0.1556642 , 0.16944568, 0.17008253, 0.17189045, 0.16371333,
         0.16920382],
        [0.15427158, 0.16910604, 0.17078262, 0.17210388, 0.16401082,
         0.16972506],
        [0.15413071, 0.17030559, 0.17129961, 0.17388957, 0.16045214,
         0.16992239],
        [0.15558615, 0.16898607, 0.17091793, 0.17271997, 0.16374463,
         0.16804524]]),
 'transition': array([[9.55547112e-01, 6.07902736e-03, 3.79939210e-03, 0.00000000e+00,
         0.00000000e+00, 3.45744681e-02],
        [5.54141696e-04, 9.84340366e-01, 1.34020195e-02, 3.48904031e-04,
         9.44093260e-04, 4.10475330e-04],
        [6.95398202e-05, 3.64504558e-03, 9.85906596e-01, 1.13581706e-03,
         4.28828891e-04,

In [65]:
import os
import tempfile
import shutil
import joblib
import tarfile

featureCols = np.array(train_features_df.columns)
# Write model to file
outFile = os.path.join("/home/aayush/accelerometer/custom_model", 'model.tar')
saveToTar(outFile,
          model=model,
          labels=labels,
          featureCols=featureCols,
          hmmParams=hmmParams)
print(f'Output trained model written to: {outFile}')

Models saved to /home/aayush/accelerometer/custom_model/model.tar
Output trained model written to: /home/aayush/accelerometer/custom_model/model.tar


In [66]:
# Display feature importances
feature_importances = pd.Series(model.feature_importances_, index=df.columns[:-4])
print(feature_importances.nlargest(30))  # Show the top 20 features

xMean          0.045498
yMean          0.039204
zMean          0.038496
avgArmAngel    0.037697
pitchg         0.037644
yawg           0.035130
rollg          0.031321
xRange         0.026077
enmoAbs        0.019157
enmoTrunc      0.018886
yRange         0.018559
zRange         0.018068
skew           0.017881
corrxy         0.017658
kurt           0.016948
avgyaw         0.016546
MAD            0.015962
entropy        0.015924
xStd           0.015363
avgroll        0.015347
avgpitch       0.015332
zStd           0.015115
corrxz         0.015056
sdyaw          0.015010
yStd           0.014439
p625           0.013943
totalPower     0.013451
p1             0.013387
p2             0.013330
corryz         0.013061
dtype: float64


In [67]:
import sklearn.metrics as metrics

# test the performance
Y_labels_test_pred = model.predict(test_features_df)
print(metrics.classification_report(test_labels_df, Y_labels_test_pred))

              precision    recall  f1-score   support

   bicycling       0.03      0.35      0.05       160
       mixed       0.32      0.39      0.35      1030
   sit-stand       0.32      0.12      0.17      1735
       sleep       0.76      0.57      0.65      2356
     vehicle       0.10      0.13      0.11       272
     walking       0.09      0.04      0.06       807

    accuracy                           0.33      6360
   macro avg       0.27      0.27      0.23      6360
weighted avg       0.44      0.33      0.36      6360



In [68]:
testScore = metrics.f1_score(test_labels_df, Y_labels_test_pred, average='macro', zero_division=0)
print(f'Score: {testScore:.2f}')

Score: 0.23


In [69]:
def viterbi(Y_obs, hmm_params):
    """ Perform HMM smoothing over observations via Viteri algorithm

    https://en.wikipedia.org/wiki/Viterbi_algorithm

    :param dict hmm_params: Dictionary containing prior, emission and transition
        matrices, and corresponding labels

    :return: Smoothed sequence of activities
    :rtype: numpy.array
    """

    def log(x):
        SMALL_NUMBER = 1e-16
        return np.log(x + SMALL_NUMBER)

    prior = hmm_params['prior']
    emission = hmm_params['emission']
    transition = hmm_params['transition']
    labels = hmm_params['labels']

    nobs = len(Y_obs)
    nlabels = len(labels)

    Y_obs = np.where(Y_obs.reshape(-1, 1) == labels)[1]  # to numeric

    probs = np.zeros((nobs, nlabels))
    probs[0, :] = log(prior) + log(emission[:, Y_obs[0]])
    for j in range(1, nobs):
        for i in range(nlabels):
            probs[j, i] = np.max(
                log(emission[i, Y_obs[j]]) +
                log(transition[:, i]) +
                probs[j - 1, :])  # probs already in log scale
    viterbi_path = np.zeros_like(Y_obs)
    viterbi_path[-1] = np.argmax(probs[-1, :])
    for j in reversed(range(nobs - 1)):
        viterbi_path[j] = np.argmax(
            log(transition[:, viterbi_path[j + 1]]) +
            probs[j, :])  # probs already in log scale

    viterbi_path = labels[viterbi_path]  # to labels

    return viterbi_path

In [70]:
YpredHmm = viterbi(Y_labels_test_pred, hmmParams)

print('\nTest performance (HMM):')
print(metrics.classification_report(test_labels_df, YpredHmm))
testHmmScore = metrics.f1_score(test_labels_df, YpredHmm, average='macro', zero_division=0)
print(f'Score: {testHmmScore:.2f}')


Test performance (HMM):
              precision    recall  f1-score   support

   bicycling       0.00      0.00      0.00       160
       mixed       0.00      0.00      0.00      1030
   sit-stand       0.00      0.00      0.00      1735
       sleep       0.37      1.00      0.54      2356
     vehicle       0.00      0.00      0.00       272
     walking       0.00      0.00      0.00       807

    accuracy                           0.37      6360
   macro avg       0.06      0.17      0.09      6360
weighted avg       0.14      0.37      0.20      6360

Score: 0.09


/home/aayush/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aayush/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aayush/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
